In [1]:
import datetime
import pandas as pd
from scipy.io import loadmat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn import metrics
import matplotlib.pyplot as plt

import numpy as np

In [2]:
#Function to load battery discharge .mat file
def load_data_discharge(battery):
  mat = loadmat('./DATA/1. BatteryAgingARC-FY08Q4/' + battery + '.mat')
  print('Total data in dataset ' + battery + ': ', len(mat[battery][0, 0]['cycle'][0]))
  counter = 0
  dataset = []
  capacity_data = []
  
  for i in range(len(mat[battery][0, 0]['cycle'][0])):
    row = mat[battery][0, 0]['cycle'][0, i]
    if row['type'][0] == 'discharge':
      ambient_temperature = row['ambient_temperature'][0][0]
      date_time = datetime.datetime(int(row['time'][0][0]),
                               int(row['time'][0][1]),
                               int(row['time'][0][2]),
                               int(row['time'][0][3]),
                               int(row['time'][0][4])) + datetime.timedelta(seconds=int(row['time'][0][5]))
      data = row['data']
      capacity = data[0][0]['Capacity'][0][0]
      for j in range(len(data[0][0]['Voltage_measured'][0])):
        voltage_measured = data[0][0]['Voltage_measured'][0][j]
        current_measured = data[0][0]['Current_measured'][0][j]
        temperature_measured = data[0][0]['Temperature_measured'][0][j]
        current_load = data[0][0]['Current_load'][0][j]
        voltage_load = data[0][0]['Voltage_load'][0][j]
        time = data[0][0]['Time'][0][j]
        dataset.append([counter + 1, ambient_temperature, date_time, capacity,
                        voltage_measured, current_measured,
                        temperature_measured, current_load,
                        voltage_load, time])
      capacity_data.append([counter + 1, ambient_temperature, date_time, capacity])
      counter = counter + 1
  #print(dataset[0])
  return [pd.DataFrame(data=dataset,
                       columns=['cycle', 'ambient_temperature', 'datetime',
                                'capacity', 'voltage_measured',
                                'current_measured', 'temperature_measured',
                                'current_load', 'voltage_load', 'time']),
          pd.DataFrame(data=capacity_data,
                       columns=['cycle', 'ambient_temperature', 'datetime',
                                'capacity'])]


#Function to load battery charge .mat file
def load_data_charge(battery):
  mat = loadmat('./DATA/1. BatteryAgingARC-FY08Q4/' + battery + '.mat')
  print('Total data in dataset ' + battery + ': ', len(mat[battery][0, 0]['cycle'][0]))
  counter = 0
  dataset = []
  
  for i in range(len(mat[battery][0, 0]['cycle'][0])):
    row = mat[battery][0, 0]['cycle'][0, i]
    if row['type'][0] == 'charge':
      ambient_temperature = row['ambient_temperature'][0][0]
      date_time = datetime.datetime(int(row['time'][0][0]),
                               int(row['time'][0][1]),
                               int(row['time'][0][2]),
                               int(row['time'][0][3]),
                               int(row['time'][0][4])) + datetime.timedelta(seconds=int(row['time'][0][5]))
      data = row['data']
      for j in range(len(data[0][0]['Voltage_measured'][0])):
        voltage_measured = data[0][0]['Voltage_measured'][0][j]
        current_measured = data[0][0]['Current_measured'][0][j]
        temperature_measured = data[0][0]['Temperature_measured'][0][j]
        current_charge = data[0][0]['Current_charge'][0][j]
        voltage_charge = data[0][0]['Voltage_charge'][0][j]
        time = data[0][0]['Time'][0][j]
        dataset.append([counter + 1, ambient_temperature, date_time, 
                        voltage_measured, current_measured,
                        temperature_measured, current_charge,
                        voltage_charge, time])
      counter = counter + 1
  #print(dataset[0])
  return pd.DataFrame(data=dataset,
                       columns=['cycle', 'ambient_temperature', 'datetime',
                                'voltage_measured',
                                'current_measured', 'temperature_measured',
                                'current_charge', 'voltage_charge', 'time'])


#Load all batteries' discharge and its measured capacity
B0005_discharge, B0005_capacity = load_data_discharge('B0005')
B0006_discharge, B0006_capacity = load_data_discharge('B0006')
B0007_discharge, B0007_capacity = load_data_discharge('B0007')
B0018_discharge, B0018_capacity = load_data_discharge('B0018')

Total data in dataset B0005:  616
Total data in dataset B0006:  616
Total data in dataset B0007:  616
Total data in dataset B0018:  319


In [3]:
#Load all batteries' charge
B0005_charge = load_data_charge('B0005')
B0006_charge = load_data_charge('B0006')
B0007_charge = load_data_charge('B0007')
B0018_charge = load_data_charge('B0018')

Total data in dataset B0005:  616
Total data in dataset B0006:  616
Total data in dataset B0007:  616
Total data in dataset B0018:  319


In [4]:
#Extract nsample data of battery's voltage, current and temperature for a specific cycle
def extract_cycle_VIT(batt, nsample, ncyl, features):
    nfeatures = len(features)
    out = batt.loc[batt['cycle']==ncyl, features]
    out = out.head(nsample)#out.sample(n = nsample)
    out = out.melt(ignore_index=True)
    out = out['value'].values.reshape(1, nfeatures*nsample, 1)
    return out

#Extract nsample data of battery's voltage, current and temperature for all cycles
def extract_VIT(batt, nsample, features):
    data = []
    temp_cyc = batt['cycle'].drop_duplicates(keep='first')
    for i in range(1, len(temp_cyc)):
        temp = extract_cycle_VIT(batt, nsample, i, features)
        data.append(temp)
    arr = np.array(data)
    return arr

#Set up training dataset of 3 batteries
def set_x_training_dataset(batt1, batt2, batt3):
    xTrain = np.append(batt1[0:len(batt1)-1], batt2[0:len(batt2)-1], axis=0)
    xTrain = np.append(xTrain, batt3[0:len(batt3)-1], axis=0)
    return xTrain

#Set up training label of 3 batteries
def set_y_training_dataset(batt1, batt2, batt3):
    yTrain = np.append(batt1, batt2, axis=0)
    yTrain = np.append(yTrain, batt3, axis=0)
    return yTrain

#Set up test dataset of 3 batteries
def set_x_test_dataset(batt):
    xTest = np.array(batt[0:len(batt)-1])
    return xTest

#Set up test label of 3 batteries
def set_y_test_dataset(batt):
    xTest = np.array(batt)
    return xTest

In [5]:
scaler = MinMaxScaler()
feature_discharge = ['capacity', 'voltage_measured', 'current_measured', 
                     'temperature_measured', 'current_load', 
                     'voltage_load', 'time']
feature_charge = ['voltage_measured', 'current_measured', 
                  'temperature_measured', 'current_charge', 
                  'voltage_charge', 'time']

B0005_discharge['cell'] = "B0005"
B0006_discharge['cell'] = "B0006"
B0007_discharge['cell'] = "B0007"
B0018_discharge['cell'] = "B0018"

B0005_charge['cell'] = "B0005"
B0006_charge['cell'] = "B0006"
B0007_charge['cell'] = "B0007"
B0018_charge['cell'] = "B0018"

cell_discharge = pd.concat([B0005_discharge, B0006_discharge, B0007_discharge, B0018_discharge])
cell_charge = pd.concat([B0005_charge, B0006_charge, B0007_charge, B0018_charge])

B0005_SOH = B0005_capacity['capacity']/B0005_capacity['capacity'][0]
B0006_SOH = B0006_capacity['capacity']/B0006_capacity['capacity'][0]
B0007_SOH = B0007_capacity['capacity']/B0007_capacity['capacity'][0]
B0018_SOH = B0018_capacity['capacity']/B0018_capacity['capacity'][0]

X_cell_discharge = scaler.fit_transform(cell_discharge[feature_discharge])
X_cell_discharge = np.insert(X_cell_discharge, 0, cell_discharge['cycle'].to_numpy(), axis=1 )
X_cell_discharge = pd.DataFrame(X_cell_discharge, columns=np.insert(feature_discharge, 0, 'cycle'))
X_cell_discharge['cell'] = cell_discharge.reset_index(drop=True)['cell']

X_cell_charge = scaler.fit_transform(cell_charge[feature_charge])
X_cell_charge = np.insert(X_cell_charge, 0, cell_charge['cycle'].to_numpy(), axis=1 )
X_cell_charge = pd.DataFrame(X_cell_charge, columns=np.insert(feature_charge, 0, 'cycle'))
X_cell_charge['cell'] = cell_charge.reset_index(drop=True)['cell']

X_B0005_discharge = X_cell_discharge.loc[X_cell_discharge['cell'] == 'B0005']
X_B0006_discharge = X_cell_discharge.loc[X_cell_discharge['cell'] == 'B0006']
X_B0007_discharge = X_cell_discharge.loc[X_cell_discharge['cell'] == 'B0007']
X_B0018_discharge = X_cell_discharge.loc[X_cell_discharge['cell'] == 'B0018']

X_B0005_charge = X_cell_charge.loc[X_cell_charge['cell'] == 'B0005']
X_B0006_charge = X_cell_charge.loc[X_cell_charge['cell'] == 'B0006']
X_B0007_charge = X_cell_charge.loc[X_cell_charge['cell'] == 'B0007']
X_B0018_charge = X_cell_charge.loc[X_cell_charge['cell'] == 'B0018']

In [6]:
#Set sample to 10
nsample = 100

#Set required features to be loaded
features = ['voltage_measured', 
            'current_measured', 
            'temperature_measured', 
            #'current_charge', 
            #'voltage_charge', 
            'cycle']

nfeatures = len(features)

#Load VIT features of all batteries dataset
dt_B0005_VIT = extract_VIT(X_B0005_charge, nsample, features)
dt_B0006_VIT = extract_VIT(X_B0006_charge, nsample, features)
dt_B0007_VIT = extract_VIT(X_B0007_charge, nsample, features)
dt_B0018_VIT = extract_VIT(X_B0018_charge, nsample, features)

#Set up the training and test dataset
xTrain = set_x_training_dataset(dt_B0005_VIT, dt_B0006_VIT, dt_B0007_VIT)
xTest = set_x_test_dataset(dt_B0018_VIT)

#Set up the training and test label
yTrain = set_y_training_dataset(B0005_SOH, B0006_SOH, B0007_SOH)
yTest = set_y_test_dataset(B0018_SOH)

In [7]:
import torch
from torch.utils.data import Dataset

class BatteryDataset(Dataset):
    def __init__(self, features, labels):
        """
        Args:
            features (numpy.ndarray or torch.Tensor): Feature matrix (e.g., xTrain or xTest).
            labels (numpy.ndarray or torch.Tensor): Labels (e.g., yTrain or yTest).
        """
        self.features = torch.tensor(features, dtype=torch.float32)  # Convert to Tensor
        self.labels = torch.tensor(labels, dtype=torch.float32)      # Convert to Tensor

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    

# Assuming xTrain, xTest, yTrain, yTest are prepared using your dataset functions
# Ensure xTrain, xTest, yTrain, and yTest are numpy arrays or tensors

# Create Dataset objects
train_dataset = BatteryDataset(xTrain, yTrain)
test_dataset = BatteryDataset(xTest, yTest)

# Create DataLoaders
batch_size = 5  # Set the desired batch size
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
np.shape(xTrain)

(504, 1, 400, 1)

In [9]:
train_dataset

In [18]:
import torch
import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self, input_shape):
        super(CNNModel, self).__init__()
        # Adjust kernel size to fit input width of 1
        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=30, kernel_size=(1, 1), stride=(1, 1))
        self.leaky_relu1 = nn.LeakyReLU(0.1)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))

        self.conv2 = nn.Conv2d(in_channels=30, out_channels=15, kernel_size=(1, 1), stride=(1, 1))
        self.leaky_relu2 = nn.LeakyReLU(0.1)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))

        # Calculate the flatten size dynamically
        self.flatten_size = self._get_flatten_size(input_shape)

        self.fc = nn.Linear(self.flatten_size, 1)

    def _get_flatten_size(self, input_shape):
        x = torch.zeros(1, *input_shape)  # Simulate a batch with size 1
        x = self.conv1(x)
        x = self.leaky_relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.leaky_relu2(x)
        x = self.pool2(x)
        return x.numel()

    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.leaky_relu2(x)
        x = self.pool2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Example usage
input_shape = (1, 400, 1)  # Shape of xTrain[1]
model = CNNModel(input_shape).to(device) 

# Print model summary
print(model)

# Example forward pass
example_input = torch.randn(1, *input_shape).to(device)  # Batch size 1
output = model(example_input)
print("Output shape:", output.shape)


Using device: cuda
CNNModel(
  (conv1): Conv2d(1, 30, kernel_size=(1, 1), stride=(1, 1))
  (leaky_relu1): LeakyReLU(negative_slope=0.1)
  (pool1): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(30, 15, kernel_size=(1, 1), stride=(1, 1))
  (leaky_relu2): LeakyReLU(negative_slope=0.1)
  (pool2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=1500, out_features=1, bias=True)
)
Output shape: torch.Size([1, 1])


In [21]:
# Model, criterion, and optimizer
input_shape = xTrain.shape[1:]  # Shape excluding batch size
model = CNNModel(input_shape)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training parameters
epochs = 200

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for features, labels in train_loader:
        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate training loss
    train_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_features, val_labels in test_loader:
            val_outputs = model(val_features)
            val_loss += criterion(val_outputs, val_labels).item()

    val_loss /= len(test_loader)

    # Print losses for this epoch
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Testing loop
test_loss = 0.0
with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features)
        test_loss += criterion(outputs, labels).item()

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")


Epoch 1/200, Train Loss: 0.5020, Val Loss: 0.0354
Epoch 2/200, Train Loss: 0.0301, Val Loss: 0.0058
Epoch 3/200, Train Loss: 0.0294, Val Loss: 0.0053
Epoch 4/200, Train Loss: 0.0257, Val Loss: 0.0042
Epoch 5/200, Train Loss: 0.0166, Val Loss: 0.0073
Epoch 6/200, Train Loss: 0.0265, Val Loss: 0.0058
Epoch 7/200, Train Loss: 0.0172, Val Loss: 0.0091
Epoch 8/200, Train Loss: 0.0275, Val Loss: 0.0080
Epoch 9/200, Train Loss: 0.0284, Val Loss: 0.0203
Epoch 10/200, Train Loss: 0.0217, Val Loss: 0.0100
Epoch 11/200, Train Loss: 0.0184, Val Loss: 0.0042
Epoch 12/200, Train Loss: 0.0209, Val Loss: 0.0023
Epoch 13/200, Train Loss: 0.0152, Val Loss: 0.0066
Epoch 14/200, Train Loss: 0.0196, Val Loss: 0.0050
Epoch 15/200, Train Loss: 0.0149, Val Loss: 0.0109
Epoch 16/200, Train Loss: 0.0160, Val Loss: 0.0062
Epoch 17/200, Train Loss: 0.0151, Val Loss: 0.0133
Epoch 18/200, Train Loss: 0.0150, Val Loss: 0.0027
Epoch 19/200, Train Loss: 0.0154, Val Loss: 0.0193
Epoch 20/200, Train Loss: 0.0153, Val Lo

In [23]:
import torch
import math

# Testing loop with RMSE and MAPE calculation
model.eval()
test_loss = 0.0
predictions = []
actuals = []

with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features)
        predictions.extend(outputs.view(-1).tolist())  # Collect predictions
        actuals.extend(labels.view(-1).tolist())      # Collect actual values
        test_loss += criterion(outputs, labels).item()

# Compute final RMSE
test_loss /= len(test_loader)
rmse = math.sqrt(sum((p - a) ** 2 for p, a in zip(predictions, actuals)) / len(actuals))

# Compute MAPE
mape = sum(abs((p - a) / a) for p, a in zip(predictions, actuals) if a != 0) / len(actuals) * 100

# Print results
print(f"Test Loss (MSE): {test_loss:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAPE: {mape:.2f}%")


Test Loss (MSE): 0.0057
Test RMSE: 0.0766
Test MAPE: 6.39%
